In [14]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

# 忽略警告
import warnings
warnings.filterwarnings('ignore')

DATA_ROOT = "./data/mid_exam/"

In [15]:
df_train = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
df_test = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))


In [16]:
df_test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,NaN,NaN,0.0,NaN
3,1439408,2632,8591.0,20:1,0.0,20160516.0
4,2029232,450,1532.0,30:5,0.0,20160530.0


In [17]:
print(df_test.shape)

(594142, 6)


In [18]:
#去除.Coupon_id=Null的資料
df_test = df_test[~df_test.Coupon_id=Null的資料.isna()]

SyntaxError: invalid syntax (<ipython-input-18-6f67a31a8b7c>, line 2)

In [19]:
df_test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,NaN,NaN,0.0,NaN
3,1439408,2632,8591.0,20:1,0.0,20160516.0
4,2029232,450,1532.0,30:5,0.0,20160530.0


In [20]:
print(df_test.shape)

(594142, 6)


In [21]:
# 一些資料被刪除了，所以重建index
df_test.reset_index(drop=True, inplace=True)

In [22]:
df_test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,NaN,NaN,0.0,NaN
3,1439408,2632,8591.0,20:1,0.0,20160516.0
4,2029232,450,1532.0,30:5,0.0,20160530.0


In [23]:
print(df_test.shape)
print(df_train.shape)

(594142, 6)
(1160742, 7)


In [24]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1 (15天內有用coupon =1)
2) buy with coupon but out of 15 days ==> 0 (超過15天才用coupon =0)
3) buy without coupon ==> -1 (we don't care) (沒有用coupon = -1)

Date_received: 優惠券取得時間
Coupon_id: 優惠券 ID (null 代表無優惠券消費)
Date: 購買商品時間 
    如果 Date is null & Coupon_id is not null, 則該紀錄為有優惠券但未使用
    若為 Date is not null & Coupon_id is null, 則為普通消費日期; 
    若 Date is not null & Coupon_id is not null, 則表示優惠券消費日期
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

df_train["label"] = df_train.apply(label, axis=1)
df_train["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [25]:
train_Y = df_train['label']
ids = df_test['User_id']
df_train = df_train.drop(['User_id', 'label'] , axis=1)
df_test = df_test.drop(['User_id'] , axis=1)
df = pd.concat([df_train, df_test])
df.head()

,Coupon_id,Date,Date_received,Discount_rate,Distance,Merchant_id
0,NaN,20160217.0,NaN,NaN,0.0,2632
1,8591.0,NaN,20160217.0,20:1,0.0,2632
2,1078.0,NaN,20160319.0,20:1,0.0,2632
3,7610.0,NaN,20160429.0,200:20,0.0,3381
4,11951.0,NaN,20160129.0,200:20,1.0,3381


In [26]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
na_check(df)

,Missing Ratio
Date,73.974975
Discount_rate,39.979964
Date_received,39.979964
Coupon_id,39.979964
Distance,6.040456


In [ ]:
#特微工程
#把欄位 Date_received變成weekday
#把欄位 Date變成weekday
#把Discount_rate變成實際可折扣金額
#把Distancee變成實際距離

In [15]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.743, Accuracy: 0.952


In [16]:
targetset = dftest.copy()
print(targetset.shape)
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)
testset = targetset[predictors].copy()

y_test_pred = model.predict_proba(testset[predictors])
test1 = testset.copy()
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

(306313, 19)
(306313, 15)


C:\Users\TOM\Anaconda3\lib\site-packages\sklearn\pipeline.py:381: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [17]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)


In [18]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
# out.to_csv("baseline_example.csv", header=["uid", "label"], index=False) # submission format
out.head()

,uid,label
0,1000020_2705_20160519,0.113585
1,1000020_8192_20160513,0.087884
2,1000065_1455_20160527,0.067504
3,1000085_8067_20160513,0.070397
4,1000086_2418_20160613,0.060909


In [19]:
out.to_csv('out.csv', index=False )